In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns
# sns.set(font_scale=4)
# import matplotlib.pyplot as plt
# %matplotlib inline
import os
import warnings
warnings.filterwarnings("ignore")

path = os.getcwd()

In [2]:
ds = pd.read_csv(os.path.join(path, 'casestudy.csv'))
ds.shape

(685927, 4)

In [3]:
ds.head()

,Unnamed: 0,customer_email,net_revenue,year
0,0,nhknapwsbx@gmail.com,249.92,2015
1,1,joiuzbvcpn@gmail.com,87.61,2015
2,2,ukkjctepxt@gmail.com,168.38,2015
3,3,gykatilzrt@gmail.com,62.40,2015
4,4,mmsgsrtxah@gmail.com,43.08,2015


In [4]:
ds = ds.drop(columns=['Unnamed: 0'])

In [5]:
ds.head()

,customer_email,net_revenue,year
0,nhknapwsbx@gmail.com,249.92,2015
1,joiuzbvcpn@gmail.com,87.61,2015
2,ukkjctepxt@gmail.com,168.38,2015
3,gykatilzrt@gmail.com,62.40,2015
4,mmsgsrtxah@gmail.com,43.08,2015


In [6]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [7]:
ds.to_sql('rev', con=engine)

685927

## Total revenue for the current year


In [8]:
tot_rev_yearly = pd.DataFrame(engine.execute("""
SELECT year, SUM(net_revenue) as yearly_revenue
    FROM rev
    GROUP BY year
    ORDER BY year
""").fetchall())

tot_rev_yearly

,year,yearly_revenue
0,2015,29036749.19
1,2016,25730943.59
2,2017,31417495.03


## New Customer Revenue e.g. new customers not present in previous year only


In [9]:
new_customer_revenue = pd.DataFrame(engine.execute(
"""
WITH prev_year_cust AS (
    SELECT DISTINCT customer_email as email, MIN(year) as first_year
    FROM rev
    GROUP BY customer_email
    ORDER BY MIN(year)
) SELECT prev_year_cust.first_year as year_, SUM(net_revenue) as new_customer_revenue
    FROM rev, prev_year_cust
    WHERE customer_email=prev_year_cust.email and year = prev_year_cust.first_year
    GROUP BY year_
"""
).fetchall())
new_customer_revenue

,year_,new_customer_revenue
0,2015,29036749.19
1,2016,18245491.01
2,2017,28676607.64


## Existing Customer Growth
To calculate this, use the Revenue of existing customers for current year –(minus) Revenue of existing customers from the previous year


In [10]:
pvt_table = pd.pivot_table(ds, values='net_revenue', index='customer_email', columns='year')
pvt_table.to_sql('pvt_table', con=engine)
pd.DataFrame(engine.execute("select * from pvt_table").fetchall())

,customer_email,2015,2016,2017
0,aaafxtkgxo@gmail.com,NaN,127.62,NaN
1,aaagldjgdt@gmail.com,44.72,NaN,NaN
2,aaaglfuvti@gmail.com,NaN,160.79,NaN
3,aaaicvtnzx@gmail.com,49.20,NaN,NaN
4,aaaiekepot@gmail.com,NaN,NaN,43.68
...,...,...,...,...
604613,zzzoxtrfic@gmail.com,NaN,NaN,15.74
604614,zzzrjsowtl@gmail.com,95.80,NaN,NaN
604615,zzztwcvgvp@gmail.com,129.75,NaN,NaN
604616,zzzwmlwwmn@gmail.com,NaN,98.01,NaN


In [11]:
exist_cust_growth = pd.DataFrame(engine.execute("""
WITH g16 AS (
    SELECT SUM([2016] - [2015]) AS diff16 FROM pvt_table WHERE [2016] IS NOT NULL AND [2015] IS NOT NULL),
g17 AS (SELECT SUM([2017] - [2016]) AS diff17 FROM pvt_table WHERE [2016] IS NOT NULL AND [2017] IS NOT NULL)
SELECT 0 AS '2015', diff16 as '2016', diff17 as '2017'
FROM g16, g17
    """).fetchall())
exist_cust_growth

# Existing customer growth is pretty stable year on year

,2015,2016,2017
0,0,20335.46,20611.34


## Revenue lost from attrition


In [12]:
rev_lost_attrition = pd.DataFrame(engine.execute("""
WITH g15 AS (SELECT SUM([2015]) AS a15 FROM pvt_table WHERE [2015] IS NOT NULL AND [2016] IS NULL),
g16 AS (SELECT SUM([2016]) AS a16 FROM pvt_table WHERE [2016] IS NOT NULL AND [2017] IS NULL)
SELECT 0 AS '2015', a15 as '2016', a16 as '2017'
FROM g16, g15
    """).fetchall())
rev_lost_attrition

# The revenue lost from attrition is almost as huge as the total revenue, indicating that revenue could have been double the current amount if customers were retained.

,2015,2016,2017
0,0,21571632.07,23110294.94


## Existing Customer Revenue Current Year


In [13]:
exist_cust_rev_curr_year = pd.DataFrame(engine.execute("""
WITH g15 AS (SELECT SUM([2015]) AS a15 FROM pvt_table),
g16 AS (SELECT SUM([2016]) AS a16 FROM pvt_table WHERE [2015] IS NOT NULL AND [2016] IS NOT NULL),
g17 AS (SELECT SUM([2017]) AS a17 FROM pvt_table WHERE [2016] IS NOT NULL AND [2017] IS NOT NULL)
SELECT a15 as '2015', a16 as '2016', a17 as '2017'
FROM  g15, g16, g17
    """).fetchall())
exist_cust_rev_curr_year

# Existing customer revenue seems to be falling down year on year

,2015,2016,2017
0,29036749.19,7485452.58,2641259.99


## Existing Customer Revenue Prior Year


In [14]:
exist_cust_rev_prior_year = pd.DataFrame(engine.execute("""
WITH g16 AS (SELECT SUM([2015]) AS a16 FROM pvt_table WHERE [2015] IS NOT NULL AND [2016] IS NOT NULL),
g17 AS (SELECT SUM([2016]) AS a17 FROM pvt_table WHERE [2016] IS NOT NULL AND [2017] IS NOT NULL)
SELECT 0 as '2015', a16 as '2016', a17 as '2017'
FROM g16, g17
    """).fetchall())
exist_cust_rev_prior_year

,2015,2016,2017
0,0,7465117.12,2620648.65


## Total Customers Current Year


In [15]:
tot_cust_curr_year = pd.DataFrame(engine.execute("""
WITH g15 AS (SELECT COUNT(customer_email) AS a15 FROM pvt_table WHERE [2015] IS NOT NULL),
g16 AS (SELECT COUNT(customer_email) AS a16 FROM pvt_table WHERE [2016] IS NOT NULL),
g17 AS (SELECT COUNT(customer_email) AS a17 FROM pvt_table WHERE [2017] IS NOT NULL)
SELECT a15 as '2015', a16 as '2016', a17 as '2017'
FROM  g15, g16, g17
    """).fetchall())
tot_cust_curr_year

# Number of customers see a dip in 2016 before rising back in 2017

,2015,2016,2017
0,231294,204646,249987


## Total Customers Previous Year


In [16]:
tot_cust_prev_year = pd.DataFrame(engine.execute("""
WITH g16 AS (SELECT COUNT(customer_email) AS a16 FROM pvt_table WHERE [2015] IS NOT NULL),
g17 AS (SELECT COUNT(customer_email) AS a17 FROM pvt_table WHERE [2016] IS NOT NULL)
SELECT 0 as '2015', a16 as '2016', a17 as '2017'
FROM g16, g17
    """).fetchall())
tot_cust_prev_year

,2015,2016,2017
0,0,231294,204646


## New Customers


In [17]:
new_custs = pd.DataFrame(engine.execute("""
WITH 
n15 AS (SELECT customer_email AS cn15 , '2015' AS year_joined FROM pvt_table WHERE [2015] IS NOT NULL),
n16 AS (SELECT customer_email AS cn16 , '2016' AS year_joined FROM pvt_table WHERE [2015] IS NULL AND [2016] IS NOT NULL),
n17 AS (SELECT customer_email AS cn17, '2017' AS year_joined FROM pvt_table WHERE [2016] IS NULL AND [2017] IS NOT NULL)
SELECT cn15 customer_emails, year_joined
FROM n15
UNION ALL
SELECT cn16, year_joined
FROM n16
UNION ALL
SELECT cn17, year_joined
FROM n17
""").fetchall())
new_custs

# The firm gained 605383 new customers from years 2015 to 2017

,customer_emails,year_joined
0,aaagldjgdt@gmail.com,2015
1,aaaicvtnzx@gmail.com,2015
2,aaaihliwuo@gmail.com,2015
3,aaaqpobaaa@gmail.com,2015
4,aaatyrafec@gmail.com,2015
...,...,...
605379,zzzdnukepj@gmail.com,2017
605380,zzzeehgitl@gmail.com,2017
605381,zzzmkaruhf@gmail.com,2017
605382,zzznqseiwv@gmail.com,2017


## Lost Customers

In [18]:
lost_custs = pd.DataFrame(engine.execute("""
WITH l16 AS (SELECT customer_email AS cl16 , '2016' AS year_left FROM pvt_table WHERE [2015] IS NOT NULL AND [2016] IS NULL),
l17 AS (SELECT customer_email AS cl17, '2017' FROM pvt_table WHERE [2016] IS NOT NULL AND [2017] IS NULL)
SELECT cl16 as customer_email, year_left 
FROM l16
UNION ALL
SELECT cl17, '2017'
FROM l17
""").fetchall())
lost_custs

# The firm has lost 355397 customers in 2016 and 2017 combined

,customer_email,year_left
0,aaagldjgdt@gmail.com,2016
1,aaaicvtnzx@gmail.com,2016
2,aaaihliwuo@gmail.com,2016
3,aabeiqckqu@gmail.com,2016
4,aabipmtctn@gmail.com,2016
...,...,...
355392,zzxqawadmp@gmail.com,2017
355393,zzylxsttdq@gmail.com,2017
355394,zzysesrqck@gmail.com,2017
355395,zzyyqigvjs@gmail.com,2017
